# Model Gateway Tests 

The model gateway provides a kafka to http bridge.

In [ ]:
!pip install kafka-python

In [1]:
from kafka import KafkaProducer
def produceIrisJson():
    key = str.encode(''.join(random.choices(string.ascii_uppercase + string.digits, k=10)))
    value='{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'
    b=str.encode(value)
    producer = KafkaProducer(bootstrap_servers=BROKER_IP+':9092')
    producer.send("seldon.default.model.iris.inputs",value=b,key=key)
    producer.flush()
    producer.close()
    return key

In [2]:
from kafka import KafkaConsumer
import random
import string

def consumeIrisJson(key):
    group_id = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
    consumer = KafkaConsumer(
         bootstrap_servers=[BROKER_IP+':9092'],
         group_id=group_id,
         auto_offset_reset="earliest",
    )
    consumer.subscribe(['seldon.default.model.iris.outputs'])
    
    try:
        for msg in consumer:
            if msg.key == key:
                print(msg.value)
                break
    finally:
        consumer.close()


In [3]:
import v2_dataplane_pb2 as v2
from google.protobuf.json_format import Parse

def produceIrisGrpc():
    key = str.encode(''.join(random.choices(string.ascii_uppercase + string.digits, k=10)))
    data = '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}'
    msg = Parse(data,v2.ModelInferRequest())
    b = msg.SerializeToString()
    producer = KafkaProducer(bootstrap_servers=BROKER_IP+':9092')
    producer.send("seldon.default.model.iris.inputs",value=b,key=key)
    producer.flush()
    producer.close()
    return key

In [4]:
from kafka import KafkaConsumer
import random
import string

def consumeIrisGrpc(key):
    group_id = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
    consumer = KafkaConsumer(
         bootstrap_servers=[BROKER_IP+':9092'],
         group_id=group_id,
         auto_offset_reset="earliest",
    )
    consumer.subscribe(['seldon.default.model.iris.outputs'])
    
    try:
        for msg in consumer:
            if msg.key == key:
                resp = v2.ModelInferResponse()
                resp.ParseFromString(msg.value)
                print(resp)
                break
   
    finally:
        consumer.close()

# Model Gateway Local Test


## Setup


* `make start-all-mlserver`
* Update the config file `config/agent.yaml` locally 
```
kafka:
  active: true
  broker: "kafka:29092"
```

Note: Seems not to presently work with confluent-kafka with Docker compose kraft (no zookeeper) Kafka.



In [5]:
BROKER_IP="0.0.0.0"

In [6]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1}\
              }}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

{
  
}


In [7]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "totalMemoryBytes": "1000000",
      "availableMemoryBytes": "1199500",
      "numLoadedModels": 1,
      "overCommitPercentage": 20
    }
  ],
  "expectedReplicas": -1,
  "availableReplicas": 1,
  "numLoadedModelReplicas": 1
}


In [8]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

*   Trying 0.0.0.0...
* Connected to 0.0.0.0 (127.0.0.1) port 9000 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: 0.0.0.0:9000
> User-Agent: curl/7.47.0
> Accept: */*
> Content-Type: application/json
> seldon-model: iris
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< content-length: 196
< content-type: application/json
< date: Sat, 26 Mar 2022 15:22:52 GMT
< server: envoy
< x-envoy-upstream-service-time: 1186
< seldon-route: iris_1
< 
* Connection #0 to host 0.0.0.0 left intact
{"model_name":"iris_1","model_version":"1","id":"75077c6a-398e-4cf3-96a7-11f537049515","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [9]:
key = produceIrisJson()
consumeIrisJson(key)

b'{"model_name":"iris_1","model_version":"1","id":"86d16f98-09c5-4572-b792-127008dcd712","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}'


In [10]:
key = produceIrisGrpc()
consumeIrisGrpc(key)

model_name: "iris_1"
model_version: "1"
outputs {
  name: "predict"
  datatype: "INT64"
  shape: 1
  contents {
    int64_contents: 2
  }
}



In [11]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}


# Model Gateway K8S Test


## Setup


* Install on k8s
* Update the config map seldon-agent 
```yaml
kafka:
    active: true
    broker: "seldon-kafka-plain-bootstrap.kafka:9092"
```

In [ ]:
BROKER_IP=!kubectl get svc seldon-kafka-plain-bootstrap -n kafka -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
BROKER_IP=BROKER_IP[0]
BROKER_IP

In [ ]:
SCHEDULER_IP=!kubectl get svc seldon-scheduler -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
SCHEDULER_IP=SCHEDULER_IP[0]
import os
os.environ['SCHEDULER_IP'] = SCHEDULER_IP
SCHEDULER_IP

In [ ]:
MESH_IP=!kubectl get svc seldon-mesh -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP
MESH_IP

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1},\
              "streamSpec":{"inputTopic":"iris-in","outputTopic":"iris-out"}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

In [ ]:
!curl -v http://${MESH_IP}:80/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
produceIrisJson()
consumeIrisJson()

In [ ]:
produceIrisGrpc()
consumeIrisGrpc()

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel